In [1]:
%load_ext py5
%gui osx

In [2]:
import pandas as pd
import numpy as np
from sciviewer import SCIViewer

In [3]:
print('DOWNLOADING AND EXTRACTING EXAMPLE DATA')
! mkdir -p ../data
! wget https://www.dropbox.com/s/ntwa1x9hxjzwwhx/pbmc3k_umap_20210420.tsv -O ../data/pbmc3k_umap_20210420.tsv
! wget https://www.dropbox.com/s/ruzzcpc4mw3hakp/pbmc3k_expression_log2TP10K_20210420.tsv -O ../data/pbmc3k_expression_log2TP10K_20210420.tsv
! ls ../data

DOWNLOADING AND EXTRACTING EXAMPLE DATA
--2021-04-20 17:34:02--  https://www.dropbox.com/s/ntwa1x9hxjzwwhx/pbmc3k_umap_20210420.tsv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ntwa1x9hxjzwwhx/pbmc3k_umap_20210420.tsv [following]
--2021-04-20 17:34:02--  https://www.dropbox.com/s/raw/ntwa1x9hxjzwwhx/pbmc3k_umap_20210420.tsv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd381d54cf1794d3b80e19309d7.dl.dropboxusercontent.com/cd/0/inline/BM9GD3eRnQ4AaLpVfkIZ5zAyoO-JPK_YnKrnqx1C96sZxloxr4rra3QdcLX_WvLKT657DXBCzCLPVUfYGOXCPf3QN686hXngDkpICoMevrte_uIubxgPqLzzF-m2GB0cW8_d8nvzAX3ZLaQiuuQhpiVB/file# [following]
--2021-04-20 17:34:02--  https://ucd381d54cf1794d3b80e19309d7.dl.dropboxusercontent.com/cd/0/inline/BM9GD3eRnQ4AaLpVfkIZ5zAyoO-JPK_YnKrnqx1C96

In [3]:
print("LOADING UMAP DATA...")

umap = pd.read_csv('../data/pbmc3k_umap_20210420.tsv', sep='\t', index_col=0)
umap.head() 

LOADING UMAP DATA...


,UMAP_1,UMAP_2
AAACATACAACCAC-1,9.728817,4.212151
AAACATTGAGCTAC-1,3.799765,10.181845
AAACATTGATCAGC-1,7.023628,4.829623
AAACCGTGCTTCCG-1,-0.298523,2.024061
AAACCGTGTATGCG-1,8.018503,-0.300426


In [4]:
print("LOADING GENE EXPRESSION DATA...")

expr = pd.read_csv('../data/pbmc3k_expression_log2TP10K_20210420.tsv', sep='\t', index_col=0)
expr.iloc[:5,:5]

LOADING GENE EXPRESSION DATA...


,AL627309.1,AP006222.2,RP11-206L10.2,RP11-206L10.9,LINC00115
AAACATACAACCAC-1,0.0,0.0,0.0,0.0,0.0
AAACATTGAGCTAC-1,0.0,0.0,0.0,0.0,0.0
AAACATTGATCAGC-1,0.0,0.0,0.0,0.0,0.0
AAACCGTGCTTCCG-1,0.0,0.0,0.0,0.0,0.0
AAACCGTGTATGCG-1,0.0,0.0,0.0,0.0,0.0


In [67]:
svobj = SCIViewer(umap, expr)
svobj.explore_data()

0.09125685691833496 seconds to select and project cells
Selected 629 cells
Calculating correlations...


/Users/dylankotliar/miniconda3/envs/sciviewer/lib/python3.8/site-packages/sciviewer/sciviewer.py:613: RuntimeWarning: invalid value encountered in true_divide
  rs = np.dot(DP, DO) / np.sqrt(np.sum(DO ** 2, 0) * np.sum(DP ** 2))


2.0089569091796875 seconds to calculate correlations. Sparsity:  False
Selected gene FCGR3A
Min/max expression level for gene FCGR3A 0.0 4.1987143
Selected gene RHOC
Min/max expression level for gene RHOC 0.0 3.7419977
Selected gene IFITM2
Min/max expression level for gene IFITM2 0.0 5.139403
Selected gene RPS19
Min/max expression level for gene RPS19 0.0 6.014548
Selected gene MS4A7
Min/max expression level for gene MS4A7 0.0 4.4668155
Selected gene S100A8
Min/max expression level for gene S100A8 0.0 5.953327
Selected gene LYZ
Min/max expression level for gene LYZ 0.0 6.460587
Selected gene S100A9
Min/max expression level for gene S100A9 0.0 6.331901


## The class attributes below get populated in real time and thus can be accessed when the interactive viewer is running

In [68]:
## This attribute get updated in real time whenever cells are selected

svobj.selected_cells.head()

,index,cell_name,projection
0,3,AAACCGTGCTTCCG-1,0.606419
1,9,AAACGCTGTTTCTG-1,0.708702
2,13,AAAGAGACGCGAGA-1,0.314478
3,16,AAAGCAGATATCGG-1,0.403731
4,22,AAAGTTTGTAGCGT-1,0.080186


In [69]:
## This gets updated in real time when cells are selected in directional mode
svobj.results_proj_correlation.sort_values(by='P').head()

,R,P
S100A8,-0.799867,3.810651e-141
FCGR3A,0.793967,1.238228e-137
LYZ,-0.788619,1.508324e-134
S100A9,-0.772039,1.587497e-125
RHOC,0.663269,5.694504e-81


0.06225109100341797 seconds to select and project cells
Selected 341 cells
Calculating differential expression...
0.03258109092712402 seconds to calculate genesums. Sparsity:  False
0.3313109874725342 seconds to calculate squared genesums. Sparsity:  False


/Users/dylankotliar/miniconda3/envs/sciviewer/lib/python3.8/site-packages/sciviewer/sciviewer.py:572: RuntimeWarning: invalid value encountered in sqrt
  remainder_stds = np.sqrt((self.gene_sqsum - selected_stds - (remainder_N*remainder_means**2)) / (remainder_N -1))


1.471858263015747 seconds to calculate differential expression. Sparsity:  False
Selected gene CD74
Min/max expression level for gene CD74 0.0 6.048452
Selected gene HLA-DRA
Min/max expression level for gene HLA-DRA 0.0 5.969516
Selected gene HLA-DPB1
Min/max expression level for gene HLA-DPB1 0.0 6.1795025
Selected gene IL32
Min/max expression level for gene IL32 0.0 5.5627327
Selected gene S100A4
Min/max expression level for gene S100A4 0.0 5.734671


In [70]:
## This gets updated in real time when cells are selected in differential mode
svobj.results_diffexpr.sort_values(by='P').head()

,T,P
HLA-DRA,74.540722,0.000000e+00
CD74,79.800984,0.000000e+00
HLA-DPB1,53.956061,2.257030e-272
HLA-DRB1,49.154586,2.278954e-243
CD3D,-36.755880,3.394393e-207


0.20721697807312012 seconds to select and project cells
Selected 1564 cells
Calculating correlations...


/Users/dylankotliar/miniconda3/envs/sciviewer/lib/python3.8/site-packages/sciviewer/sciviewer.py:613: RuntimeWarning: invalid value encountered in true_divide
  rs = np.dot(DP, DO) / np.sqrt(np.sum(DO ** 2, 0) * np.sum(DP ** 2))


2.8474719524383545 seconds to calculate correlations. Sparsity:  False
Selected gene RPL32
Min/max expression level for gene RPL32 0.0 5.7993245
Selected gene RPS12
Min/max expression level for gene RPS12 0.0 6.1948423
EXPORTING DATA...
BYE


## The class attributes below only get populated when the export and close button is pushed

In [ ]:
svobj.significant_genes.head()